In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False

In [2]:
import sys
import os

In [3]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

True

In [5]:
%cd VideoPrediction_MovingMNIST/

/content/VideoPrediction_MovingMNIST


In [4]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")):
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..

    %pip install -qr requirements.txt  --quiet
    setup_already_done = True

Cloning into 'VideoPrediction_MovingMNIST'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 112 (delta 58), reused 84 (delta 33), pack-reused 0
Receiving objects: 100% (112/112), 55.15 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/VideoPrediction_MovingMNIST
/content/VideoPrediction_MovingMNIST/data
/content/VideoPrediction_MovingMNIST
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9

In [6]:
# import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,random_split
# from torch import optim, nn, utils, Tensor
# import torchvision
# from torchvision.transforms import ToTensor
import lightning.pytorch as pl

In [7]:
# !nvidia-smi

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")
train_set, validation_set, test_set = random_split(dataset,[8000, 1000, 1000],
                                                   generator=torch.Generator().manual_seed(42))
# Visualization Test
dataset.visualize(0)

In [10]:
loader = DataLoader(dataset)
training_dataloader = DataLoader(train_set)
validation_dataloader = DataLoader(validation_set)
test_dataloader = DataLoader(test_set)

In [11]:
from src.simpleLSTM import simpleLSTM
import src.Modules as mod
from src.parameters import C


model = simpleLSTM()


encoder = mod.Encoder(C,3)
decoder = mod.Decoder(C,3)
model_eddy = mod.PlEncoderDecoder(encoder,decoder)

In [12]:
# for data in loader:
#     print(model.training_step(data,0))
#     break

In [ ]:
%pip install wandb

In [13]:
from lightning.pytorch.loggers.wandb import WandbLogger
logger = WandbLogger()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
# trainer = pl.Trainer(limit_train_batches=1, max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, limit_train_batches=100, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=5, limit_train_batches=100)
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", logger=logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model=model, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | relu    | ReLU       | 0     
2 | encoder | Sequential | 4.2 M 
3 | lstm    | LSTM       | 3.1 M 
4 | decoder | Sequential | 4.7 M 
---------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.275    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | relu    | ReLU       | 0     
2 | encoder | Sequential | 4.2 M 
3 | lstm    | LSTM       | 3.1 M 
4 | decoder | Sequential | 4.7 M 
---------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.275    Total estimated model par

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [16]:
trainer.test(model, dataloaders=test_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     2640.734130859375     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2640.734130859375}]

In [19]:
# !zip -r ../logs lightning_logs/

In [17]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/ #--port 6007

In [18]:
# import gc
# print(torch.cuda.list_gpu_processes())
# gc.collect()
# torch.cuda.empty_cache()